In [ ]:
import sys  # noqa
import os
import subprocess
import logging
import warnings
from multiprocessing import Manager, Pool, Process
import pickle as pkl

import numpy as np
import pandas as pd
from pandarallel import pandarallel
import seaborn as sns
import matplotlib.pyplot as plt
from statannotations.Annotator import Annotator
from scipy import stats

from downstream.rna_seq import volcano, convert_ensg_to_symbol, pyPPI, pyTCGA, plot_gene,plot_boxplot_gene,get_contrast
from downstream.editing_reditools import get_dict_reditools, plot_site_counts, get_samp_df, plot_editing_overview_shared_site, plot_editing_overview_all_site, get_melt_df_outer, plot_editing_events_pk_read, get_file_info, plot_editing_events, get_REDIT_count_df, check_editing_group, get_samp_df_frequency
warnings.filterwarnings("ignore", category=DeprecationWarning)

THREADS = 60
os.environ["POLARS_MAX_THREADS"] = str(THREADS)
logging.basicConfig(
    stream=sys.stdout,
    format="%(asctime)s %(levelname)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO)
logger = logging.getLogger(__name__)
logger.info("work_dir: %s" % os.getcwd())

def do_something_background(myfunction, *args, **kwargs):
    my_process = Process(target=myfunction, args=args, kwargs=kwargs)
    my_process.daemon = True
    my_process.start()

def do_some_command_background(command, log_file=None):
    if log_file is None:
        log_file = open('log_file.log', 'a')
    my_process = Process(
        target=subprocess.run,
        kwargs={
            'args': command,
            'shell': True,
            'stderr': log_file,
            'stdout': log_file})
    my_process.daemon = True
    my_process.start()

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats
inference = DefaultInference(n_cpus=16)

In [2]:
columns = []
expression_count = ''
expression_tpm = ''
meta_data = pd.DataFrame({'Condition': ['ctl',
                                        'ctl',
                                        'kd',
                                        'kd',
                                        ]},
                         index=columns)


res_path = ''
reditools_res_path = ''
file_info_path = ''
aei = ''
name = ''
os.chdir(res_path)

reditools_res_t = get_dict_reditools(reditools_res_path)
res_downsample_t = get_dict_reditools(reditools_res_path + '_downsample')
file_info = get_file_info(file_info_path)
file_info_downsample = get_file_info(file_info_path + '_downsample')

reditools_res = {}
res_downsample = {}
for i in columns:
    reditools_res[i] = reditools_res_t[i]
    res_downsample[i] = res_downsample_t[i]

In [8]:
s293t_ctl_1 = pd.read_csv(
    '293T-ctl-1.reditools2.annotated.tsv.gz',
    sep='\t')
s293t_ctl_2 = pd.read_csv(
    '293T-ctl-2.reditools2.annotated.tsv.gz',
    sep='\t')
s293t_kd_1 = pd.read_csv(
    '293T-kd-1.reditools2.annotated.tsv.gz',
    sep='\t')
s293t_kd_2 = pd.read_csv(
    '293T-kd-2.reditools2.annotated.tsv.gz',
    sep='\t')

In [ ]:
def df_to_list(s293t_kd_1):
    s293t_kd_1 = s293t_ctl_1[(s293t_ctl_1.FILTER_PASS)
                             & (s293t_ctl_1.REDIPortal)]
    return list(s293t_kd_1['CHROM'].astype(str) + ':' + s293t_kd_1['POS'].astype(str))
len(set(df_to_list(s293t_kd_1) + df_to_list(s293t_kd_2) + df_to_list(s293t_ctl_1) + df_to_list(s293t_ctl_2))) 

In [ ]:
%run common_editing.ipynb